In [46]:
# use natural language toolkit
import nltk
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
# word stemmer
stemmer = LancasterStemmer()

"It is important to note that for Multinomial Naive Bayes you will need enough data ideally not more that 10^4 (10,000) sentences with  various classes to increase it's accuracy"

In [17]:
# 3 classes of training data
training_data = []
training_data.append({"class":"greeting", "sentence":"how are you?"})
training_data.append({"class":"greeting", "sentence":"how is your day?"})
training_data.append({"class":"greeting", "sentence":"good day"})
training_data.append({"class":"greeting", "sentence":"how is it going today?"})

training_data.append({"class":"goodbye", "sentence":"have a nice day"})
training_data.append({"class":"goodbye", "sentence":"see you later"})
training_data.append({"class":"goodbye", "sentence":"have a nice day"})
training_data.append({"class":"goodbye", "sentence":"talk to you soon"})

training_data.append({"class":"MSBLQ", "sentence":"what is meristem stock brokers"})
training_data.append({"class":"MSBLQ", "sentence":"what should i buy?"})
training_data.append({"class":"MSBLQ", "sentence":"where is meristem located?"})
training_data.append({"class":"MSBLQ", "sentence":"is meristem stockbrokers a licensed company?"})
print ('%s sentences of training data ' % len(training_data))

12 sentences of training data 


In [5]:
#lets try to tekenize and stem the words into different varaibles
corpus_word = {}
class_word = {}

#lets bring out the attribute from the key-'class'
classes = list(set([a['class'] for a in training_data]))

#lets place the words in 'a' into the the dictionary class_word
for c in classes:
    class_word[c] = []

#lets pull out the words in the sentence to tekenize and stem
for data in training_data:
    
    #lets tokenize the data
    for word in nltk.word_tokenize(data['sentence']):
        
        #lets remove '?' and 's'
        if word not in ['?', 's']:
            
            #lets stem the words now
            stemmed_word = stemmer.stem(word.lower())
            
            #lets see how many times the words occur and sum the count
            if stemmed_word not in corpus_word:
                corpus_word[stemmed_word] = 1
            else:
                corpus_word[stemmed_word] += 1
                
            #lets insert the stemmed words in the class words 
            class_word[data['class']].extend([stemmed_word])
            
#lets print the words and its counts and also the the class and the words under them
print ('corpus words and count: %s \n' % corpus_word)
print ('class words: %s' % class_word)        

corpus words and count: {'how': 3, 'ar': 1, 'you': 3, 'is': 5, 'yo': 1, 'day': 4, 'good': 1, 'it': 1, 'going': 1, 'today': 1, 'hav': 2, 'a': 3, 'nic': 2, 'see': 1, 'lat': 1, 'talk': 1, 'to': 1, 'soon': 1, 'what': 2, 'meristem': 3, 'stock': 1, 'brok': 1, 'should': 1, 'i': 1, 'buy': 1, 'wher': 1, 'loc': 1, 'stockbrok': 1, 'licens': 1, 'company': 1} 

class words: {'MSBLQ': ['what', 'is', 'meristem', 'stock', 'brok', 'what', 'should', 'i', 'buy', 'wher', 'is', 'meristem', 'loc', 'is', 'meristem', 'stockbrok', 'a', 'licens', 'company'], 'greeting': ['how', 'ar', 'you', 'how', 'is', 'yo', 'day', 'good', 'day', 'how', 'is', 'it', 'going', 'today'], 'goodbye': ['hav', 'a', 'nic', 'day', 'see', 'you', 'lat', 'hav', 'a', 'nic', 'day', 'talk', 'to', 'you', 'soon']}


In [6]:
#lets create a new sentence to see how accurate the prediction is
sentence = 'how do i get to know about meristem stockbrokers?'

#lets create a function to claculate the class score
def calculate_class_score(sentence, c, show_details = True):
    
    #lets create an empty score 
    score = 0
    
    #lets tokenize and stem the words
    for word in nltk.word_tokenize(sentence):
        
        #lets stem the words
        if stemmer.stem(word.lower()) in class_word[c]:
            
            #add a to the score
            score += 1
        #else:
            #score = 1
        
        #show the match   
            if show_details:
                print ('  Match: %s' % stemmer.stem(word.lower()))
    return score


In [7]:
#lets get the class and the score
for c in class_word.keys():
    print ('Class: %s Score: %s \n' % (c, calculate_class_score(sentence, c)))

  Match: i
  Match: meristem
  Match: stockbrok
Class: MSBLQ Score: 3 

  Match: how
Class: greeting Score: 1 

  Match: to
Class: goodbye Score: 1 



In [8]:
#create a function to calculate the score for commonality of words in a class
def calculate_class_score_commonality(sentence, c, show_details = True):
    
    #calculate a score
    score = 0
    
    #just as done earlier,lets tokenize and stem the sentence created earlier
    
    #lets tokenize and stem the words
    for word in nltk.word_tokenize(sentence):
        
        #lets stem the words
        if stemmer.stem(word.lower()) in class_word[c]:
            
            #lets score the words reducing the words with commonality
            score += (1 / corpus_word[stemmer.stem(word.lower())])
            
            #lets match the score to the words
            if show_details:
                print (' Match: %s (%s)' % (stemmer.stem(word.lower()), 1 / corpus_word[stemmer.stem(word.lower())]))
    return score

In [9]:
#lets get the class and the commonality score
for c in class_word.keys():
    print ('Class: %s Score: %s \n' % (c, calculate_class_score_commonality(sentence, c)))

 Match: i (1.0)
 Match: meristem (0.3333333333333333)
 Match: stockbrok (1.0)
Class: MSBLQ Score: 2.333333333333333 

 Match: how (0.3333333333333333)
Class: greeting Score: 0.3333333333333333 

 Match: to (1.0)
Class: goodbye Score: 1.0 



In [24]:
#lets define a function that will give the highest class and score
def classify(sentence):
    
    #lets create variables for the high score and class
    high_class = None
    high_score = 0
    
    #lets create a loop for to pick out our class
    for c in class_word.keys():
        
        #lets make a variable to put our previous commonality score claculation
        score = calculate_class_score_commonality(sentence, c, show_details = False)
        
        #lets make a conditional statement to put the result in the high class and score
        if score > high_score:
            high_score = score
            high_class = c
            
    return high_class, high_score
    

In [15]:
classify('david')

(None, 0)

In [16]:
classify('where is meristem brokers situated?')

('MSBLQ', 2.533333333333333)

In [18]:
classify('good day')

('greeting', 1.25)

In [20]:
classify('how is your day going')

('greeting', 2.783333333333333)

In [27]:
classify('hope you are well?')

('greeting', 1.3333333333333333)